In [1]:
import smtplib
import os
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase

In [2]:
def send_email(smtp_info, msg):
    with smtplib.SMTP(smtp_info['smtp_server'], smtp_info['smtp_port']) as server:
        # TLS 보안 설정 및 시작(보안 정책상 필수)
        server.starttls()
        
        # 로그인
        server.login(smtp_info['smtp_user_id'], smtp_info['smtp_user_pw'])
        
        # 이메일 전송
        # 이메일 전송 시 반드시 as_string()으로 변환
        res = server.sendmail(msg['from'], msg['to'], msg.as_string())
        
        if not res:
            print('이메일 전송 성공')
        else:
            print(res)

In [3]:
def make_multimsg(msg_dict):
    multi = MIMEMultipart(_subtype = 'mixed')
    
    for key, value in msg_dict.items():
        # text 형식인 경우
        if key == 'text':
            with open(value['filename'], encoding='utf-8') as fp:
                msg = MIMEText(fp.read(), _subtype = value['subtype'])
        
        # image 형식인 경우
        elif key == 'image':
            with open(value['filename'], 'rb') as fp:
                msg = MIMEImage(fp.read(), _subtype = value['subtype'])
                
        # audio, video 형식은 설정x
        elif key == 'audio':
            print('음원은 지원하지 않습니다.')
        elif key == 'video':
            print('영상은 지원하지 않습니다.')
            
        # 그 외의 형식    
        else:
            with open(value['filename'], 'rb') as fp:
                msg = MIMEBase(value['maintype'], _subtype = value['subtype'])
                msg.set_payload(fp.read())
                encoders.encode_base64(msg)
        
        # 파일 이름은 첨부파일 제목
        msg.add_header('Content-Disposition', 'attachment',
                      filename = os.path.basename(value['filename']))
        
        # 파일 첨부
        multi.attach(msg)
        
    return multi

In [6]:
# text 와 image 함께 전송하기
smtp_info = dict({
    'smtp_server': 'smtp.naver.com',
    'smtp_user_id': '@naver.com',
    'smtp_user_pw': '',
    'smtp_port': 587
})

msg_dict = {
    # text형식의 첨부파일
    'text': {
        'maintype': 'text',
        'subtype': 'plain',
        'filename': 'email/test.txt'
    },
    # image형식의 첨부파일
    'image': {
        'maintype': 'image',
        'subtype': 'jfif',
        'filename': 'email/보노보노.jfif'
    },
    # pdf형식의 첨부파일
    'application': {
        'maintype': 'application',
        'subtype': 'octect-stream',
        'filename': 'email/test.pdf'
    }  
}

# 메일 전송
title = '첨부파일 포함하여 메일보내기'
content = '첨부파일을 확인해보세요.'
sender = smtp_info['smtp_user_id']
receiver = smtp_info['smtp_user_id']

msg = MIMEText(_text = content, _charset = 'utf-8')

# 파일 첨부
multi = make_multimsg(msg_dict)
multi['Subject'] = title
multi['From'] = sender
multi['To'] = receiver
multi.attach(msg)

send_email(smtp_info, multi)

이메일 전송 성공
